## import data and display head 

In [1]:
%%time
import pandas as pd
import numpy as np

url1 = "https://datasets.imdbws.com/title.basics.tsv.gz"
url2 = "https://datasets.imdbws.com/title.akas.tsv.gz"
url3 = "https://datasets.imdbws.com/title.ratings.tsv.gz"

basics = pd.read_csv(url1, sep='\t', low_memory=False)
akas = pd.read_csv(url2, sep='\t', low_memory=False)
ratings = pd.read_csv(url3, sep='\t', low_memory=False)

CPU times: total: 4min
Wall time: 5min 40s


In [2]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [3]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [4]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1916
1,tt0000002,5.8,259
2,tt0000003,6.5,1722
3,tt0000004,5.6,172
4,tt0000005,6.2,2540


## Lets clean title.basics dataset

In [5]:
%%time
# Replace '\N' with np.nan
basics.replace({'\\N':np.nan}, inplace=True)

# Drop any rows that have null values in either genre or runtimeMinutes.
basics.dropna(subset=['runtimeMinutes', 'genres'], inplace=True)

# Drop all rows where titleType!=Movie. Keep only titleType==Movie
is_Movie = basics['titleType'] == 'Movie'
basics = basics[is_Movie]

# Drop all rows where the start year is not between 2000-2022
# keep startYear 2000-2022
filter2000 = basics['startYear'] >= 2000 
filter2022 = basics['startYear'] <=2022
basics = basics[filter2000 & filter2022]

# Exclude movies that are included in the documentary category.
is_documentary = basics['genres'].str.contains('documentary',case=False)
basics = basics[~is_documentary]

# Filter the basics table down to only include the US by using the filter akas dataframe
keepers = ratings['tconst'].isin(akas['titleId'])
basics = basics[keepers]
basics.head()

C:\Users\default.DESKTOP-0LDO0LD\AppData\Local\Temp\ipykernel_3096\4118908317.py:23: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  basics = basics[keepers]


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


## Lets clean akas

In [6]:
akas.replace({'\\N':np.nan}, inplace=True)
keepers = ratings['tconst'].isin(akas['titleId'])
akas = akas[keepers]

C:\Users\default.DESKTOP-0LDO0LD\AppData\Local\Temp\ipykernel_3096\2665373585.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  akas = akas[keepers]


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

## Lets clean ratings

In [8]:
ratings.replace({'\\N':np.nan}, inplace=True)
keepers = ratings['tconst'].isin(akas['titleId'])
ratings = ratings[keepers]

## save cleaned files in Data folder

In [ ]:
## Save current dataframe to file.
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
akas.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)